In [1]:
# necessary libraries
import numpy as np
import pandas as pd
import plotly.express as px
import dash
from dash import dcc, html, Input, Output

In [2]:
df = pd.read_csv('IPL_2022_Sold_Players.csv')
df.head()

,Players,Nationality,Type,Price Paid,Team
0,Robin Uthappa,Indian,Batsman,"2,00,00,000",CSK
1,Dwayne Bravo,Overseas,All-Rounder,"4,40,00,000",CSK
2,Ambati Rayudu,Indian,Wicket Keeper,"6,75,00,000",CSK
3,Deepak Chahar,Indian,Bowler,"14,00,00,000",CSK
4,C.Hari Nishaanth,Indian,Batsman,"20,00,000",CSK


In [3]:
df.shape

(204, 5)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204 entries, 0 to 203
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Players      204 non-null    object
 1   Nationality  204 non-null    object
 2   Type         204 non-null    object
 3   Price Paid   204 non-null    object
 4   Team         204 non-null    object
dtypes: object(5)
memory usage: 8.1+ KB


In [5]:
# Price Paid column is object type that need to be changed into int
df['Price Paid'] = df['Price Paid'].str.replace(',', '').astype(int)

In [6]:
df.Team.nunique()

10

In [7]:
df.Team.value_counts()

Team
PK     23
CSK    21
KKR    21
MI     21
RR     21
DC     20
GT     20
SRH    20
RCB    19
LSG    18
Name: count, dtype: int64

In [8]:
fig = px.histogram(df,x='Team', title='Number of Player Each Team Purchased')

fig.update_xaxes(title_text='Team Name')
fig.update_yaxes(title_text='No. of Players')

fig.show()

In [9]:
s = df.groupby('Team')['Price Paid'].agg(sum)
s

C:\Users\Pias\AppData\Local\Temp\ipykernel_14356\2174704478.py:1: FutureWarning:

The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.



Team
CSK    450500000
DC     474000000
GT     518500000
KKR    475500000
LSG    590000000
MI     479000000
PK     685500000
RCB    554500000
RR     610500000
SRH    679000000
Name: Price Paid, dtype: int32

In [10]:
fig1 = px.pie(df, names='Team', values='Price Paid'
       , title='% Team Used in Total Spent')
fig1.show()

In [11]:
grouped_data = df.groupby(['Team', 'Type']).size().reset_index(name='Count')
grouped_data.head()


,Team,Type,Count
0,CSK,All-Rounder,7
1,CSK,Batsman,4
2,CSK,Bowler,8
3,CSK,Wicket Keeper,2
4,DC,All-Rounder,8


In [12]:
# Define your custom color map
color_map = {'Batsman': '#3498db', 'Bowler': '#2ecc71', 'Wicket Keeper':'#e74c3c','All-Rounder':'#f39c12'}

fig2 = px.bar(data_frame=grouped_data,
       x='Team',
       y='Count',
       color='Type',
       color_discrete_map=color_map,
       title='Different Types of Players in Each Team'
)

fig2.update_xaxes(title_text='Team Name')
fig2.update_yaxes(title_text='No of Players of Each Type')

fig2.show()


In [13]:
s = df['Price Paid'].max()

In [14]:
max_price_player_name = df.loc[df['Price Paid'].idxmax(), 'Players']

In [15]:
df[df['Price Paid']==df['Price Paid'].max()]['Players']

100    Ishan Kishan
Name: Players, dtype: object

### Dashboard Building

In [16]:
# Create a Dash web application
app = dash.Dash(__name__)

# Define the layout
app.layout = html.Div([
    html.H1("Cricket Player Dashboard", style={'text-align': 'center'}),

    # Filters
    html.Div([
        html.Label("Select Country"),
        dcc.Dropdown(
            id='country-dropdown',
            options=[{'label': country, 'value': country} for country in df['Nationality'].unique()] + [
                {'label': 'Overall', 'value': 'Overall'}],
            value=None,  # df['Nationality'].unique()[0]
            style={'width': '50%', 'margin-left': '15px'}
        ),

        html.Label("Select Type"),
        dcc.Dropdown(
            id='type-dropdown',
            options=[{'label': player_type, 'value': player_type} for player_type in df['Type'].unique()] + [
                {'label': 'Overall', 'value': 'Overall'}],
            value=None,
            style={'width': '50%'}
        ),

        html.Label("Select Team"),
        dcc.Dropdown(
            id='team-dropdown',
            options=[{'label': team, 'value': team} for team in df['Team'].unique()] + [
                {'label': 'Overall', 'value': 'Overall'}],
            value=None,
            style={'width': '50%'}
        ),
    ], style={'width': '100%', 'display': 'flex', 'justify-content': 'space-between', 'margin-bottom': '20px'}),

    # Charts and Cards
    html.Div([
        # Highest Paid Player Card
        html.Div([
            html.Div([
                html.H3("Highest Paid Player"),
                html.P(id="highest-paid-player")
            ], className="card", style={'width': '50%', 'float': 'left', 'margin-left': '15px'}),

            html.Div([
                html.H3("Lowest Paid Player"),
                html.P(id="lowest-paid-player")
            ], className="card", style={'width': '50%', 'float': 'right'}),
        ], style={"display": "flex", "justify-content": "space-between", 'margin-bottom': '20px'}),
    ]),

    # Histogram
    html.Div([
        dcc.Graph(id='histogram', figure=fig, style={'width': '100%', 'margin-top': '20px', 'margin-bottom': '20px'}),
    ], style={'width': '100%', 'margin-top': '20px', 'margin-bottom': '20px'}),

    # Pie chart and Bar chart on the same row
    html.Div([
        dcc.Graph(id='pie-chart', figure=fig1, style={'width': '50%', 'float': 'left'}),
        dcc.Graph(id='bar-chart', figure=fig2, style={'width': '50%', 'float': 'right'}),
    ], style={"display": "flex", "justify-content": "space-between"})
], style={'width': '100%', 'margin': 'auto'})


# Define callback to update the charts based on filters
@app.callback(
    [Output("histogram", "figure"),
     Output("pie-chart", "figure"),
     Output("bar-chart", "figure"),
     Output("highest-paid-player", "children"),
     Output("lowest-paid-player", "children")],
    [Input("country-dropdown", "value"),
     Input("type-dropdown", "value"),
     Input("team-dropdown", "value")]
)
def update_charts_and_highest_paid_player(selected_country, selected_type, selected_team):
    filtered_df = df.copy()

    if selected_country and selected_country != 'Overall':
        filtered_df = filtered_df[filtered_df['Nationality'] == selected_country]

    if selected_type and selected_type != 'Overall':
        filtered_df = filtered_df[filtered_df['Type'] == selected_type]

    if selected_team and selected_team != 'Overall':
        filtered_df = filtered_df[filtered_df['Team'] == selected_team]

    if filtered_df.empty:
        return fig, fig1, fig2, "No data for the selected filters", "No data for the selected filters"

    # Update histograms
    updated_histogram = px.histogram(filtered_df, x='Team', title='Number of Players Each Team Purchased')
    updated_histogram.update_xaxes(title_text='Team Name')
    updated_histogram.update_yaxes(title_text='No. of Players')

    # Update pie chart
    updated_pie_chart = px.pie(filtered_df, names='Team', values='Price Paid', title='% Team Used in Total Spent')

    # Update bar chart
    grouped_data = filtered_df.groupby(['Team', 'Type']).size().reset_index(name='Count')
    updated_bar_chart = px.bar(data_frame=grouped_data, x='Team', y='Count', color='Type', color_discrete_map=color_map,
                               title='Different Types of Players in Each Team')
    updated_bar_chart.update_xaxes(title_text='Team Name')
    updated_bar_chart.update_yaxes(title_text='No of Players of Each Type')

    # Update highest and lowest paid players
    max_price_player_name = filtered_df.sort_values(by='Price Paid', ascending=False).iloc[0]['Players']
    min_price_player_name = filtered_df.sort_values(by='Price Paid').iloc[0]['Players']

    return updated_histogram, updated_pie_chart, updated_bar_chart, max_price_player_name, min_price_player_name


if __name__ == '__main__':
    app.run_server(debug=True, host='127.0.0.1', port=8000)